# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming (Files)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("tarea-Files1") \
    .master("spark://fc91669459e3:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/28 18:14:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Creación del FileStream

In [ ]:
from team_name.spark_utils import SparkUtils

students_schema = SparkUtils.generate_schema([("name", "string")])

students_df = spark \
                .readStream \
                .schema(students_schema) \
                .format("json") \
                .option("cleanSource", "archive") \
                .option("sourceArchiveDir", "/home/jovyan/notebooks/data/final_log") \
                .load("/home/jovyan/notebooks/data/strucutred_streaming_files/")

In [10]:
from team_name.spark_utils import SparkUtils
from pyspark.sql.functions import split

log_schema = SparkUtils.generate_schema([("timestamp", "string"),("level","string"),("message","string"),("server","string")])

logs_df = spark \
                .readStream \
                .schema(log_schema) \
                .format("text") \
                .option("cleanSource", "archive") \
                .option("sourceArchiveDir", "/home/jovyan/notebooks/data/final_log") \
                .option("delimiter", "|") \
                .csv("/home/jovyan/notebooks/data/strucutred_streaming_files/")






In [9]:
df = spark.read.schema(log_schema).option("delimiter", "|").csv("/home/jovyan/notebooks/data/strucutred_streaming_files/server_logs.txt")
df.show(truncate=False)

+--------------------+-------+---------------------------+--------------+
|timestamp           |level  |message                    |server        |
+--------------------+-------+---------------------------+--------------+
|2025-03-28 08:16:13 | ERROR | 500 Internal Server Error | server-node-1|
|2025-03-28 08:16:16 | ERROR | 404 Not Found             | server-node-2|
|2025-03-28 08:16:19 | ERROR | 404 Not Found             | server-node-3|
|2025-03-28 08:16:20 | INFO  | User login successful     | server-node-3|
|2025-03-28 08:16:24 | INFO  | User login successful     | server-node-2|
|2025-03-28 08:16:29 | WARN  | User login successful     | server-node-3|
|2025-03-28 08:16:33 | INFO  | User login successful     | server-node-2|
|2025-03-28 08:16:35 | ERROR | 404 Not Found             | server-node-2|
|2025-03-28 08:16:37 | INFO  | User login successful     | server-node-2|
|2025-03-28 08:16:41 | ERROR | 404 Not Found             | server-node-1|
|2025-03-28 08:16:45 | INFO  | User lo

### Configuración del "Sink" del stream

In [13]:
query = logs_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='5 seconds') \
                .format("console") \
                .start()

query.awaitTermination()

25/03/28 18:22:38 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e12c477d-e276-4a06-b49f-811ebf80d921. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/28 18:22:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-------+--------------------+--------------+
|           timestamp|  level|             message|        server|
+--------------------+-------+--------------------+--------------+
|2025-03-28 12:22:34 |  WARN | User login succe...| server-node-1|
|2025-03-28 12:22:38 | ERROR |      404 Not Found | server-node-3|
+--------------------+-------+--------------------+--------------+



25/03/28 18:23:00 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/strucutred_streaming_files/server_logs0.txt to file:/home/jovyan/notebooks/data/final_log/home/jovyan/notebooks/data/strucutred_streaming_files/server_logs0.txt / skip moving file.


-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+------+--------------------+--------------+
|           timestamp| level|             message|        server|
+--------------------+------+--------------------+--------------+
|2025-03-28 12:22:57 | INFO | User login succe...| server-node-1|
|2025-03-28 12:23:00 | WARN | User login succe...| server-node-3|
+--------------------+------+--------------------+--------------+



25/03/28 18:23:20 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/strucutred_streaming_files/server_logs1.txt to file:/home/jovyan/notebooks/data/final_log/home/jovyan/notebooks/data/strucutred_streaming_files/server_logs1.txt / skip moving file.


...

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-------+-----------------+--------------+
|           timestamp|  level|          message|        server|
+--------------------+-------+-----------------+--------------+
|2025-03-28 12:23:37 | ERROR | 502 Bad Gateway | server-node-2|
+--------------------+-------+-----------------+--------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-------+-----------------+--------------+
|           timestamp|  level|          message|        server|
+--------------------+-------+-----------------+--------------+
|2025-03-28 12:23:37 | ERROR | 502 Bad Gateway | server-node-2|
+--------------------+-------+-----------------+--------------+



25/03/28 18:24:00 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/strucutred_streaming_files/server_logs3.txt to file:/home/jovyan/notebooks/data/final_log/home/jovyan/notebooks/data/strucutred_streaming_files/server_logs3.txt / skip moving file.
java.io.FileNotFoundException: File file:/home/jovyan/notebooks/data/strucutred_streaming_files/server_logs3.txt does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:390)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:341)
	at org.apache.hadoop.fs.RawLocalFileSystem.rename(RawLocalFileSystem.java:505)
	at org.apache.hadoop.fs.ChecksumFileSystem.rename(ChecksumFileSystem.java:700)
	at org.apache.hadoop.hiv

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+------+--------------------+--------------+
|           timestamp| level|             message|        server|
+--------------------+------+--------------------+--------------+
|2025-03-28 12:23:58 | WARN | User login succe...| server-node-1|
|2025-03-28 12:23:59 | INFO | User login succe...| server-node-1|
+--------------------+------+--------------------+--------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+------+--------------------+--------------+
|           timestamp| level|             message|        server|
+--------------------+------+--------------------+--------------+
|2025-03-28 12:23:58 | WARN | User login succe...| server-node-1|
|2025-03-28 12:23:59 | INFO | User login succe...| server-node-1|
+--------------------+------+--------------------+--------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
query.stop()

In [ ]:
sc.stop()